In [10]:
from pynq import Overlay
Proc_RISCV = Overlay('/home/xilinx/pynq/overlays/design_RISCVEIRB/design_RISCVEIRB.bit')
Proc_RISCV?

In [11]:
RISCV = Proc_RISCV.My_RISCVEIRB_IP_0 
RISCV?


In [12]:
from pynq import MMIO
import numpy as np
import datetime
import os

class RISCVEIRB_Controller:

    
    def __init__(self, BASE_ADDRESS = 0x43C00000, ADDRESS_LENGTH = 0x64000):
        print("Création de la class RISCVEIRB_Controller")
        self.mmio = MMIO(BASE_ADDRESS, ADDRESS_LENGTH)


    ########## METHODES ##########
        
    ## WRITE ##

    def write_inst_mem_from_tab(self, mem_instruction, size = 32):
        tableauOctets = np.array([0x00, 0x00, 0x00, 0x00])
        CODE_RAM_SIZE = size
        print("\nInstruction Memory Write Process\n\r")
        for i in range(0, CODE_RAM_SIZE, 1):
            # Envoie octets par octets
            for j in range (0, 4, 1):
                adresse = (4 * i) + j
                tableauOctets[j]  = (mem_instruction[i] >> (j*8)) % 256;
                data_ins = ((0 << 24) + (tableauOctets[j] << 16) + 0b0000000000001010); # Inst_Boot & Inst_RW_Boot
                # Ecriture dans la mémoire octets par octets
                self.mmio.write(0x04, adresse)
                self.mmio.write(0x00, int(data_ins))
            print("Octet value Write for current Instruction",hex(mem_instruction[i]),"at address", i,": [",hex(tableauOctets[3]),";",hex(tableauOctets[2]),";",hex(tableauOctets[1]),";",hex(tableauOctets[0]),"]")

    def write_data_mem_from_tab(self, mem_data, size = 32):
        tableauOctets = np.array([0x00, 0x00, 0x00, 0x00])
        print("\nData Memory Write Process\n\r")
        CODE_RAM_SIZE = size
        for i in range(0, CODE_RAM_SIZE, 1):
            for j in range (0, 4, 1):
                temp = (j  * 8)
                adresse = (4 * i) + j
                tableauOctets[j]  = (mem_data[i]>> temp)%256;
                data_val = (tableauOctets[j] << 24) + ((0 << 16) +  + 0b0000000000010100);
                self.mmio.write(0x08, adresse)    
                self.mmio.write(0x00, int(data_val))
            print("Octet value Write for current Instruction",hex(mem_data[i]),"at address", i,": [",hex(tableauOctets[3]),";",hex(tableauOctets[2]),";",hex(tableauOctets[1]),";",hex(tableauOctets[0]),"]")
    

    ## READ ##

    def read_inst_mem(self, size = 32, log_opt = False, file_name = "inst_mem"):
        tableauOctets = np.array([0x00, 0x00, 0x00, 0x00])
        mem_instruction = np.zeros(size)
        CODE_RAM_SIZE = size
        #Instruction Memory Read Process  
        print("\nInstruction Memory Read Process\n\r")
        for i in range(0, CODE_RAM_SIZE, 1):
            for j in range (0, 4, 1):
                # Sequence lecture
                # Inst_RW_Boot <= 0 (READ)
                # Inst_Boot <= 1 
                self.mmio.write(0x00,0b00000000000000000000000000000010)
                adresse = ((4 * i) + j)
                self.mmio.write(0x04, adresse)
                tableauOctets[j] = self.mmio.read(0xC)>>16
                data_ins_rd = (tableauOctets[3] << 24) + (tableauOctets[2] << 16) + (tableauOctets[1] << 8) + (tableauOctets[0]) 
            if (data_ins_rd < 0) : 
                data_ins_rd = (data_ins_rd + (1<<32))
            print("La valeur de l'instruction", i," est :", hex(data_ins_rd))
            mem_instruction[i] = data_ins_rd
         # Enregistrement dans un fichier log    
        if (log_opt):
            now = datetime.datetime.now()
            time = now.strftime("%Y-%m-%d_%H-%M-%S-%f")  # Ajout de %f pour les microsecondes
            log_file_name = "./log/"+time+"_"+file_name+".hex"
            with open(log_file_name, 'w') as f:
                for value in mem_instruction:
                    f.write(f'0x{int(value):08X},\n')
                   
            

    
    def read_data_mem(self, size = 32, log_opt = False, file_name = "data_mem"):
        tableauOctets = np.array([0x00, 0x00, 0x00, 0x00])
        mem_data = np.zeros(size)
        CODE_RAM_SIZE = size
        #Data Memory Read Process  
        print("\nInstruction Memory Read Process\n\r")
        for i in range(0, CODE_RAM_SIZE, 1):
            for j in range (0, 4, 1):
                # Data_RW_Boot <= 0 (READ)
                # Data_Boot <= 1 
                self.mmio.write(0x00,0b00000000000000000000000000000100)
                adresse = ((4 * i) + j)
                self.mmio.write(0x08, adresse)
                tableauOctets[j] = self.mmio.read(0xC)>>24
            data_val_rd = (tableauOctets[3] << 24) + (tableauOctets[2] << 16) + (tableauOctets[1] << 8) + (tableauOctets[0]) 
            if (data_val_rd  < 0) : 
                data_val_rd  = (data_val_rd  + (1<<32))
            mem_data[i] = int(data_val_rd)
            print("La valeur de la donnée", i," est :", hex(data_val_rd))
        # Enregistrement dans un fichier log    
        if (log_opt):
            now = datetime.datetime.now()
            time_extension = now.strftime("%Y-%m-%d_%H-%M-%S-%f")  # Ajout de %f pour les microsecondes
            filename = "./log/"+time_extension+"_"+file_name+".hex"
            with open(filename, 'w') as f:
                for value in mem_data:
                    f.write(f'0x{int(value):08X},\n')

    
    
#################################################################################################
                        
#################################################################################################

def charger_fichier(path):
    if path.endswith('.hex'):
        with open(path, 'r') as file:
            data = np.array([int(line.strip().rstrip(','), 16) for line in file])
    else:
        print("Format de fichier non pris en charge.")
        return None
    return data

In [13]:
print("Hello")

Hello


In [14]:
Controller = RISCVEIRB_Controller()

Création de la class RISCVEIRB_Controller


In [15]:
mem_instruction = np.array([
    0x00d00513, 
    0x00c00713, 
    0x00700593, 
    0x01c00293, 
    0xff600f93, 
    0x7e600913, 
    0x00058633, 
    0x00c02023, 
    0x00a676b3, 
    0x00d02223, 
    0x00e6e7b3, 
    0x00f02423, 
    0x0057c833, 
    0x01002623, 
    0x40b80833, 
    0x01002823, 
    0x00a838b3, 
    0x01102a23, 
    0x00581333, 
    0x00602c23, 
    0x01f323b3, 
    0x00702e23, 
    0x01233e33, 
    0x03c02023, 
    0x00535eb3, 
    0x03d02223, 
    0x40535f33, 
    0x03e02423, 
    0x000000ef,    
    0x0,
    0x0,
    0x0])

mem_data = np.array([
    0x11,
    0x7, 
    0xFCFFFE80, 
    0x0f0e0d0c,
    0x13121110, 
    0x17161514, 
    0x1b1a1918, 
    0x1f1e1d1c, 
    0x23222120, 
    0x27262524,
    0x2b2a2928, 
    0x2f2e2d2c, 
    0x76543210, 
    0xfedcba98, 
    0xffffffff, 
    0xffffffff, 
    0xffffffff,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,    
    0x0])


In [16]:
Controller.write_inst_mem_from_tab(mem_instruction=mem_instruction, size=32)
Controller.read_inst_mem(size = 32, log_opt=True)


Instruction Memory Write Process

Octet value Write for current Instruction 0xd00513 at address 0 : [ 0x0 ; 0xd0 ; 0x5 ; 0x13 ]
Octet value Write for current Instruction 0xc00713 at address 1 : [ 0x0 ; 0xc0 ; 0x7 ; 0x13 ]
Octet value Write for current Instruction 0x700593 at address 2 : [ 0x0 ; 0x70 ; 0x5 ; 0x93 ]
Octet value Write for current Instruction 0x1c00293 at address 3 : [ 0x1 ; 0xc0 ; 0x2 ; 0x93 ]
Octet value Write for current Instruction 0xff600f93 at address 4 : [ 0xff ; 0x60 ; 0xf ; 0x93 ]
Octet value Write for current Instruction 0x7e600913 at address 5 : [ 0x7e ; 0x60 ; 0x9 ; 0x13 ]
Octet value Write for current Instruction 0x58633 at address 6 : [ 0x0 ; 0x5 ; 0x86 ; 0x33 ]
Octet value Write for current Instruction 0xc02023 at address 7 : [ 0x0 ; 0xc0 ; 0x20 ; 0x23 ]
Octet value Write for current Instruction 0xa676b3 at address 8 : [ 0x0 ; 0xa6 ; 0x76 ; 0xb3 ]
Octet value Write for current Instruction 0xd02223 at address 9 : [ 0x0 ; 0xd0 ; 0x22 ; 0x23 ]
Octet value Writ

In [17]:
mem_inst_2 = charger_fichier(path="./log/2022-10-07_05-25-50-686401_inst_mem.hex")
Controller.write_data_mem_from_tab(mem_inst_2)


Data Memory Write Process

Octet value Write for current Instruction 0xd00513 at address 0 : [ 0x0 ; 0xd0 ; 0x5 ; 0x13 ]
Octet value Write for current Instruction 0xc00713 at address 1 : [ 0x0 ; 0xc0 ; 0x7 ; 0x13 ]
Octet value Write for current Instruction 0x700593 at address 2 : [ 0x0 ; 0x70 ; 0x5 ; 0x93 ]
Octet value Write for current Instruction 0x1c00293 at address 3 : [ 0x1 ; 0xc0 ; 0x2 ; 0x93 ]
Octet value Write for current Instruction 0xff600f93 at address 4 : [ 0xff ; 0x60 ; 0xf ; 0x93 ]
Octet value Write for current Instruction 0x7e600913 at address 5 : [ 0x7e ; 0x60 ; 0x9 ; 0x13 ]
Octet value Write for current Instruction 0x58633 at address 6 : [ 0x0 ; 0x5 ; 0x86 ; 0x33 ]
Octet value Write for current Instruction 0xc02023 at address 7 : [ 0x0 ; 0xc0 ; 0x20 ; 0x23 ]
Octet value Write for current Instruction 0xa676b3 at address 8 : [ 0x0 ; 0xa6 ; 0x76 ; 0xb3 ]
Octet value Write for current Instruction 0xd02223 at address 9 : [ 0x0 ; 0xd0 ; 0x22 ; 0x23 ]
Octet value Write for c